In [ ]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy.engine.url import URL
from dotenv import load_dotenv
import os
import google.generativeai as genai
from langchain_community.agent_toolkits import create_sql_agent
# from google import genai
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
server = os.getenv("MS_SQL_SERVER", ".\SQLEXPRESS")
database = os.getenv("MS_SQL_DATABASE", "DEV_RMR_ASGSecurityDB")
username = os.getenv("MS_SQL_USER", "your_username")
password = os.getenv("MS_SQL_PASSWORD", "your_password")
driver = os.getenv("MS_SQL_DRIVER", "ODBC Driver 17 for SQL Server")
google_api_key = os.environ["GOOGLE_API_KEY"]
model_name = os.getenv("gpt_deployment_name")

In [ ]:
print("Environment variables are loaded:", load_dotenv())
print("test by reading a variable:", google_api_key)

In [ ]:
import urllib

password_enc = urllib.parse.quote_plus(password)
driver_enc = urllib.parse.quote_plus(driver)

In [ ]:
# db_uri = f"mssql+pyodbc://{username}:{password_enc}@{server}/{database}?driver={driver_enc}"
db_uri = f"mssql+pyodbc://@{server}/{database}?driver={driver_enc}"

db = SQLDatabase.from_uri(
    db_uri,
    # include_tables=['Employee', 'Office', 'OfficeTeam', 'OfficeTeamEmployee']
)


In [ ]:
print(db_uri)
print(len(db.get_usable_table_names()))

In [ ]:
genai.configure(api_key=google_api_key)
llm = ChatGoogleGenerativeAI(
    model=model_name,
    google_api_key=google_api_key,
    temperature=0.0
)

In [ ]:
agent = create_sql_agent(
    llm, 
    db=db,
    agent_type="openai-tools",
    verbose=True,
    handle_parsing_errors=True,
    top_k=10
)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

question = widgets.Text(
    placeholder='',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='100%')
)
display(question)

In [ ]:
response = agent.invoke({"input": question.value})

In [ ]:
import re
output_text = response["output"]
full_text = ""

for item in response["output"]:
    if isinstance(item, dict) and "text" in item:
        full_text += item["text"]
    elif isinstance(item, str):
        full_text += item

clean_text = re.sub(r'[*]+', '', full_text)
clean_text = re.sub(r'\s*\n\s*', '\n', clean_text)
clean_text = clean_text.strip()

print(clean_text)

In [ ]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm, db)

response = chain.invoke({"question": question.value})

try:
    sql_query = response.split("SQLQuery:")[1].strip()
except IndexError:
    sql_query = response.strip()

print(f"Generated SQL: {sql_query}")

In [ ]:
result = db.run(sql_query)
print(f"Query Result: {result}")